In this notebook I will try to read pdf files and extract text from them.

I have a bunch of pdf files and I would like to extract some text data from them , here is how I will handle that

In [2]:
from pathlib import Path

In [3]:
import fitz as pdf_extractor

In [4]:
PDF_FILES_PATH = Path.cwd().parent.joinpath('data' , 'raw', 'pdf')

In [5]:
all_pdf_files =  [x for x in PDF_FILES_PATH.glob('*.pdf') if x.is_file()]

In [6]:
all_pdf_files

[PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/Manuel_in_lingala.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/LOLENGE EKOLO CONGO EZALI KOMILENGELE PE EKANI KOSALELA REDD+ (RPP) Bomoko_ Mosala _ Tombwa - PDF Free Download.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/Microsoft Research Participation Form ML.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/CV_a-first-look-pamphlet-Lingala.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/lingala.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/fcpa-lingala.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/Album village assaini Lingala.pdf'),
 PosixPath('/Users/es.py/Projects/Personal/multilingual-drc-news-chatbot/data/raw/pdf/lingala-

In [7]:
manuel_in_lingala_path  = all_pdf_files[0]

In [29]:
def parse_text_from_html(path):
    """
    parse html text from the document passed in parameter
    """
    with pdf_extractor.open(path) as doc:
        raw_text = ""
        for page in doc:
            content = page.get_text("text")
            raw_text = f"{raw_text} \n{content}"
        return raw_text

In [30]:
def write_output(path, content, name):
    with open(path.joinpath(name), "w") as output_file:
        output_file.write(f"{content}\n")

In [31]:
OUTPUT_PATH = PDF_FILES_PATH.parent.joinpath("text_from_pdf")

In [32]:
OUTPUT_PATH.mkdir(exist_ok=True)

In [33]:
OUTPUT_PATH.name

'text_from_pdf'

In [34]:
for file in all_pdf_files:
    raw_text = parse_text_from_html(file)
    file_name = file.name.__str__().replace('pdf', 'txt')
    write_output(OUTPUT_PATH, raw_text, file_name)

Manuel_in_lingala.txt
LOLENGE EKOLO CONGO EZALI KOMILENGELE PE EKANI KOSALELA REDD+ (RPP) Bomoko_ Mosala _ Tombwa - PDF Free Download.txt
Microsoft Research Participation Form ML.txt
CV_a-first-look-pamphlet-Lingala.txt
lingala.txt
fcpa-lingala.txt
Album village assaini Lingala.txt
lingala-faqs-4-8.txt
mul.txt
FORMULAIRE_BILINGUE_FR_LINGALA_FINAL.txt
LIN58-0720M By Faith Moses VGR.txt


### Cleaning the file

In this step we will try to clean the annodate corpus to make it a cleanned corpus , we will again use polygot to sptlit text into sentence and use some regular expressions to make our text data cleanned

In [35]:
TEXT_PATH = OUTPUT_PATH.joinpath('partially_cleanned')

<generator object Path.glob at 0x1100cc518>

All the files are differents in the formating but here are some steps we can perform to clean them all in one
-  remove number in a staring lines
- use polyglot to split the data into sentences
- remove (a) (B) and other similar sentence in the begining of sentences
- we can also remove urls 

In [41]:
all_files = TEXT_PATH.glob('*.txt')
with open(OUTPUT_PATH.joinpath('all.txt'), "w") as outfile:
    for file in all_files:
        with open(file, "r") as infile:
            outfile.write(f"{infile.read()}\n")

In [80]:
def remove_urls(text):
    text = re.sub(r'(https://)?(www\.)?:\/\/.*[\r\n]*', " ", text) # need to remove also url with ()
    return text

In [77]:
import re

In [90]:
raw_text = ''
with open(OUTPUT_PATH.joinpath('all.txt'), 'r') as infile:
    for line in infile.readlines():
        line = remove_urls(line)
        line = re.sub(r'^(\d\.){1,}', '', line) # stuftf like 1.1 1.3
        line = re.sub(r'^\d{1,}', '', line) # line starting with digit 1
        line = re.sub(r'^\({1,}\w+\)','', line)#(i), (A)
        line = re.sub(r'^\({1,}\w+\)','', line) # samething twicet
        line = re.sub(r'^\W-\s', '', line)# remove any non white space char ignoring line
        line = re.sub(r'^•', '', line)
        line = re.sub(r'^-', '', line)
        raw_text += f"{line}\n"
        #print(line)
        #print(line)

In [83]:
text_string =  '- Kosolola na libota na lisalisi ya moninga moko ya libota ya babuki mibeko '

In [84]:
re.sub(r'^\W_-\s', '', text_string)

'- Kosolola na libota na lisalisi ya moninga moko ya libota ya babuki mibeko '

In [96]:
from polyglot.detect.base import UnknownLanguage
from polyglot.detect import Detector
from polyglot.text import Text

In [97]:
CLEAN_FILE_PATH = OUTPUT_PATH.joinpath('all_cleanned.txt')

In [111]:
def polygot_cleaning(text):
    try:
        detector = Detector(text)
        language = detector.language.code
        print(language)
        if language == 'ln':
            sentences = Text(text).sentences
            print(len(sentences))
            for sentence in sentences:
                print(sentence)
                if len(sentence.split(' ')) >= 3:
                    with open(CLEAN_FILE_PATH, "a") as output_file:
                        output_file.write(f"{sentence.string}\n")
    except UnknownLanguage:
        print(text)

In [112]:
def read_file(path):
    with open(filepath , 'r') as file:
        content = file.read()
        return content

In [113]:
polygot_cleaning(raw_text)

ln
7005
Makambo makomami na buku oyo ya sika ya R-PP Congo masembolami na ndenge ya polele polele
Epusi bozongeli mbala na mbala matangi masusu to milongo na yango po na biteni bisusu oyo eulani na:
Mataki ya koyangela REDD+;
Tina ya makambo eutaka na bokati banzete pambapamba pe kobebisa bazamba;
Kosangisa makambo ya motuya oyo euti na makanisi ya IIASA pe eulani na oyo ya CONGOBIOM bakisa masengi ma linaka ya REDD+ ya bikolo bia Africa ya katikati oyo bazwaka lisalisi na misolo mia FEM (GEF), oyo Banque Mondiale na COMIFAC bapesaka.
MRV ya CONGO po na kosalela makinisi oyo ebongisamaki po na MRV ya ba mboka ba katikati to MRV elongo na FAO na misolo oyo CBFF apesaka;
«budget »mobimba na ba «budget» misusu oyo etali biteni binso.
Soki tososoli malamu, makambo mingi esi esalami.
Yango wana tozali kosenga na molongo mobimba bandima lolenge na biso ya kosalela REDD+ po ete basalisa biso po na makanisi na mayele oyo tosikongolo na kombo ya ekolo Congo , epusi na makambo etali boyangeli ba

maye bosalaka ememaka to makambo mabe ya bokengi, to bobebisi bokengi
na bino.
Likambo esengeli liboso ezali koluka koyeba esika bokaneli ezali kou-
ta, masengami na yango, ndenge na yango mpe soki ekoki kokokisama to te.
Likambo esengeli yango wana mpo na kozala na likanisa ya nivo ya bokaneli.
Etape ya misato etali mpe makambo mabe ya bokengi.
Makambo mabe ya
bokengi emonanaka te na eloko ebombami ya likama., kasi ebongi kozwa yan-
go lokola elembo ya bokengi na bino.
Makambo ya bokengi oyo ebongisami
mpo na moto to lingomba songolo, oyo ekangemi mbala moko na mosala na
bino, ekangemaka makasi na bokaneli mpe na esika eutaka.
Ezali bongo kati
ya esika mpe ya bizaleli to ya ekwela.
Nzokande makambo mabe ya bokengi
(ndakisa, kobungisa mafungola) ekangemi mpenza na bokaneli te.
Kasi, eko-
tungisa mpe bokengi na bino (elingi koloba soki nabungisi mafungola na ngai,
moto mosusu akoki kozwa yango mpo na kokota na bilo na ngai.)
Na etape ya minei, tokotala malamu botau mpe makoki na bino mp

Etape ya minei ezali koluka esika bokaneli ezali kouta.
Kosala boye, koluka koyeba
nani azali kotinda maloba ya bokaneli.
Bomikundola ete yango ekangemi ntango
nyonso ya mosala na bino.
Na suka, na etape ya mitano koluka kosukisa malamu mpo na koyeba soki bokaneli
oyo ekoki kosalema.
Na ntembe, ezali koluka koyeba maye ekoya lobi te, kasi komanyola na ndenge ya
mayele.
Tozali kokebisa bino na eloko oyo, etape ya yambo emonisamaka na bikela,
ezali mpenza ya bikela.
Bokoki koyeba bikela yango.
Bitape ya mibale mpe ya mis-
ato, emonisakama na bikela mwa ndambo.
Na etape ya misato mpo na koleka na
oyo ya minei, ezali bobele maloba na maloba.
Tozali kokanisa kopesa ndakisa na
bikela emonanaka mpenza.
Tokoki kokanisa ntina ya bokaneli lokola mpe esika ek-
outa.
Kasi bosukisi yango ezali bobele likambo ya makanisi.
Esengeli kokanisa oyo
esalema to ekoki kosalema.
Yango wana esengeli ntango nyonso koluka kobanda
na bikela eleka mpe emonanaki, mpe kolobaloba mpamba te na ebandeli.
Esengeli
komi

na lombangu, bopesi esika mosusu ya kofanda na lombangu, lisungi na mabota,
bolandeli mpe bosungi na lombangu, lisungi ya bosololi na telefone ntango nyonso
na babundeli ya makoki ya bato (24/24 mpe 7/7);
lisungi mpo na eleko mwa molai, bakisa mpe bolandeli ezalela ya babundeli ya
makoki ya bato, bokebisi likama noki, bokolisi makoki, nsango mpo na kokebisa
makama mpe bokengi (bakisa mpe bokengi ya odinatele), bosambeli na mokili, na
etuka mpe na ekolo mpo na likambo ya moto moko mpe mpo na likambo etali ya
mibeko, bofandisi moto esika mosusu mpo na mwa ntango mpe lisungi liboso ya
bisambiselo mpo na bosambi oyo ekoumela mingi;
lisungi mpo na eleko molai, bakisa mpe lisungi kati ya ekolo, bosambeli mpe
boluki mayele mpo na kopekisa bokutoli to bokisi bonsomi ya babundeli ya makoki
ya bato mpe kopekisa bitumbu oyo bikolo epesi bango, bakisa mpe bosambwisi,
mibeko etali bozwi misolo euti na bapaya mpe bopesi babundeli ya makoki ya bato
bitumbu.
Adelesi: contact@protectdefenders.eu
.
Life

Mabe ekómi pene, pe tobondeli
ete Ókoka kopesa biso makoki ya kosala koleka liboso.
Pémísá
banzoto na biso elembi, Nkolo, pe tíndá biso lisusu na etumba.
Mpo tosengi yango na Nkombo na Nkolo Yesu, pe na ntina na Ye
nde tobondeli.
Amen.
Tófungola Biblia na ntongo ya lelo, na—na mokapo ya 11 na
Buku ya Baebele.
Mokolo ya misato na mpokwa tolobelaki mokapo ya 7 ya
Buku ya Baebele, likolo na “Melekisedeke, oyo azali na tata te,
mama te, azali na ebandeli na mikolo te, to suka na bomoi te.”
Bongo nakanisaki, ntango mosusu, na ntongo ya lelo, ekozala
malamu kofungola lisusu Buku kitoko oyo, lokola tozali na
mobokó ya likambo tolingi kolobela, na oyo totángaki liboso.
Pe
topumbwa mokapo ya 10, ná mokapo ya 9, oyo elobeli mibeko
ya mbeka, tolingi kokóma na likambo ya “kondima”.
Pe awa, na
Buku ya Baebele, na mokapo ya 11, kobanda na molongo ya 23,
totángí Yango boye:
Na kondima Mose, ntango abotamaki, baboti na ye
babombaki ye sanza misato, mpo bamonaki ete mwana
yango azalaki kitoko; pe baban

osengeli kokangama na yango.
Luká na Makomi mpo óyeba soki
ezali ndenge wana.
“Oh,” alobi, “kendáká nde kuna.
Bango—bango basengeli
kosala bongo te kuna.
Omoni?
Kuna basalaka bongo te.”
Ndenge
wana nde, “kendé kuna,” kasi alingi te ete óbima na mboka na ye.
Ndenge wana nde Zabolo asalaka.
Alingaka te ete bótika biloko
ya mokili; bókótisa nde mokili na lingomba.
Mokolo moko, ntango nazalaki koya, nafungolaki radió
na ngai.
Ba—batiaki loyembo moko, nazalaki koyoka yango,
nasengelaki koyoka loyembo yango pene na mobimba, mpo na
koyeba soki ezalaki mpenza loyembo ya Nzambe, to Zabolo
LILOBA LILOBAMI
nde azali koluka kokitisa makambo ya Nzambe na mosándá ya
mokili.
Bokoki kosala bongo te!
Nzambe áyokela bino mawa!
Ata Elvis Presley abimisi ba-album boni, nyonso ya ba-
nzembo kitoko ya Nzambe, etali ngai te.
Akonzami bobele na
Zabolo.
Atíndí bana mingi koleka na lífelo, na misala nyonso oyo
ngai nayebi, na mokili mobimba lelo.
Pat Boone ná bamosusu,
azaláká kosambela na lingomba ya Klisto; n

yango , mpamba te manaka mango makosalemaka bipai binso
mpe ezali mpasi mpo ya bakonzi koyeba ba nani bazali o milulu.
Soki milulu ya bato ebele mpe manaka masusu ya polele to ya
esika yoko ebundisami, manaka masusu ma kotelemela o politiki
to manaka ya bisika binso mpe eye bato ba kosangana te ndakisa
koboya kofuta mpako, to koboya mosala , bikoki kozala ba nzela
malamu mpo ya lingomba.
Mangomba ma koboya ya bana mboka makoki mpe kosalela
manaka ma sika maye ma kobalola minioko mizongela baye ba
bandaki miango.
Ndakisa, kosalela ba jolona mpo ya kolakisa bilili
mpe kopesa mayebisi mpo ya kolakisa polele bizaleli mabe ma
minioko ekoki kobongola makambo mpo ya mbanda-O bikolo ba
bampaya mpe bakokata misolo.
Minioko minso te eko zongelaka
baye basaleli yango, kasi ekoki kokata bosungi eye bazalaki na
buango o ti ya ekolo to o molongo.
O ba ndakisa ebele ya koboya ya bana mboka , epusi bongo o
ekolo Filipini o mobu 1986, o ekolo Chili o mobu 1988, o ekolo
Serbia o mobu 2000 mpe o ekolo Uk

bamoyen yakolandela,
mpona kosalisa mutu wana ya wana mpona kozwa to kobomba misala mpona to elongo na, to kotinda
misala na, mutu nioso; to
mutu nioso, oyo ayebi ete nioso to ndambo ya mbongo wana to eloko ya valele ekolakisama,
ekopesama, to ekolakama, libela-libela to nakolanda sima, na mokonzi nioso ya bapaya, na lingomba
ya politiki nioso ya bapaya, to mokonzi, banda wana, to moluki mosala ya lingomba ya politiki nioso
ya bapayana, na batina ya—
Kotia bokasi na acte nioso to mokano nioso ya mokonzi ya bapaya wana, lingomba ya politiki,
mokonzi ya parti, to moluki mosala mwa parti na bokonzi bwaye moko to bwa mosala leta, (ii) koluka
kotinda mokonzi ya mopaya wana, lingomba ya politiki wana, mokonzi wana ya parti, to moluki
mosala asala to azangisa kosala acte nioso ekobuka mubeko ya mosala ya leta ya mokonzi wana ya
mopaya, lingomba ya politiki, mokonzi ya parti, to moluki mosala, to (iii) koluka kozwa ba-avantage
nioso na ndenge elongobani te; to
Koluka komeka mokonzi ya mopaya, 

eza na se):
✓ permis de conduire
✓ Passeport
✓ Carte d'identité ya kibosoda
✓ Carte d'identité ya Etuka to ya
leta
Mokanda oyo ezangi foto ekoki kozala (okoki
kotya mokanda moko na kati ya mikanda oyo
neti mokanda ya mibale):
✓ facture ya sika oyo eza na kombo na
yo pe addrèsse ya ndako na yo
✓ Carte ya sécurité sociale
✓ Certificat ya mbotama
Basango oyo eza na mikanda okotinda esengeli koyokana na oyo eza na mikanda na biso.
Mosali moko kati ya police
oyo ayebi mosala akotalela mikanda wana na kolanda ndenge azweli yango.
Osengeli kozwa mokanda ya
confirmation sima ya kokotisa mikanda na yo.
Soki basilisi kotalela identité, bakolekisa mokanda na yo ya bosengi
pe bakofuta yo sima ya mikolo kati na 7-14.
Programe ya lisungi etali kozanga mosala, Programe etali Kobatela Lifuti (PPP),
Kofungwama lisusu
.
Ebongo soki nazozwa lifuta na nzela ya PPP oyo bapesi/badefisi mokonzi na ngaï?
A.
Soki ozozwa lifuti na nzela ya PPP, osengeli kokoma na mokanda nayo ya mpono nyoso mosolo wana ndenge
e

By end of the day the text is not well cleanned , we cand do better next time.